# 导入相关包

In [1]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime,timedelta
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
import collections
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")

from tgrocery import Grocery
import jieba

In [34]:
from fuzzywuzzy import fuzz
from src.time_extractor import TimeFinder

# PDF解析原始数据 
## 加载数据并采用pdfplumber抽取PDF中的文字和表格


In [2]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = 'datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  'datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [3]:
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)

val_df = train_df[:1800]
# val_df = train_df[:1800].head(20)
train_df = train_df[1800:]
# test_df=test_df.head(20)

train_outputs["sample_id"]=train_outputs["sample_id"].astype(str)
val_df["sample_id"]=val_df["sample_id"].astype(str)
train_df["sample_id"]=train_df["sample_id"].astype(str)
test_df["sample_id"]=test_df["sample_id"].astype(str)

In [4]:
#text=text.replace(r'[ ]+',' ').replace('\r','^')
def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
 
    for item in title_num_char:
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()\[\]（）【】：:]*?[\^]")
        #pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        #pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()[]（）【】][]")#*?[\^]
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])

    # for item in title_list:
    for item in s_title_num_char:
        # pattern = re.compile(item+r"[ ]*?[ ]*?[\d][^ ]+?:?[ ]?") 
        # pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5\^ ]*")
        # pattern = re.compile(item+'[0-9\u4e00-\u9fa5()[]（）【】*?[\^]]')*?[\^]
    #    pattern = re.compile(item+r"[\d]*?\u4e00-\u9fa5+[ ]")
        pattern = re.compile(item+"[0-9A-Za-z\u4e00-\u9fa5 ()\[\]（）【】：:]*?[ ][\^]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

#from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\n","^").replace(r'[ ]+',' ')
    title_df=get_title(text)
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)`
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    index=0
    neg_index=[]
    for title_des in title_1_df[0].values:
        for item in title_neg_words:
            if re.search(item,title_des) is not None:
                neg_index.append(index)
                break
        index+=1


    return title_1_df.drop(neg_index)
    # print(title_list)

def get_judge_title_result(val_df):

    judge_title_result=None


    for sample_id,text in tqdm(val_df[["sample_id","text"]].values):
        # print(sample_id)
        # print(text)
        judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])

        judge_title_result["sample_id"]=judge_title_result["sample_id"].astype(str)        
    return judge_title_result

title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]
title_neg_words=[]
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","到期[^日].*[^2][^0]","到期.{0,2}[/^]","截至","意见","十二个月内","公告前","报备文件","前期"]

val_judge_title_result=get_judge_title_result(val_df)
train_judge_title_result=get_judge_title_result(train_df)
test_judge_title_result=get_judge_title_result(test_df)
# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)

100%|██████████| 8660/8660 [01:50<00:00, 78.18it/s]


In [5]:
###9月11日
#val_judge_title_result.head(100)
def get_content_df(val_judge_title_result):
    sample_id=val_judge_title_result['sample_id'].drop_duplicates() 
    text_list=[]
    text_sampleid_list=[]
    content_df=pd.DataFrame()
    for i in tqdm(sample_id):
        text_df=val_judge_title_result[val_judge_title_result['sample_id']==i]
    #  text_df.iloc[0,5]
        #text_df
        n=len(text_df)
        text_sampleid_list.append(i)
        text=''
        for y in range(0,n):
            text=text+text_df.iloc[y,1]+text_df.iloc[y,5]
            text=re.sub("[ ]+"," ",text).replace("（","(").replace("）",")")
        #text
        text_list.append(text)
    content_df["sample_id"]=text_sampleid_list
    content_df["text"]=text_list
    return content_df.reset_index(drop=True)


val_content_df=get_content_df(val_judge_title_result)
train_content_df=get_content_df(train_judge_title_result)
test_content_df=get_content_df(test_judge_title_result)

val_content_df=pd.merge(val_content_df,val_df[["sample_id","tabel"]],on="sample_id")
train_content_df=pd.merge(train_content_df,train_df[["sample_id","tabel"]],on="sample_id")
test_content_df=pd.merge(test_content_df,test_df[["sample_id","tabel"]],on="sample_id")

100%|██████████| 8660/8660 [00:20<00:00, 432.13it/s]


### 1.采用LSTM网络用提取好的部分跟pdf中的text做交互预测
#### 理财类型、资金来源、实际购买公司和上市公司关系、买卖方是否有关联关系

In [26]:
val_df=val_content_df
train_df=train_content_df
test_df=test_content_df

In [27]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段
train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
train_lstm_input = train_lstm_input.fillna('否')
# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
label_2 = LabelEncoder()
label_3 = LabelEncoder()
label_4 = LabelEncoder()

train_data = pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)+"_"+train_lstm_input['实际购买公司名称'].astype(str)

train_data['text_2'] = train_lstm_input['text'].astype(str)
# train_data['text_2'] = train_lstm_input['实际购买公司'].astype(str)

train_data['label_1'] = label_1.fit_transform(train_lstm_input['理财类型'])
train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])


In [28]:
# 导入相关库
import os
import pandas as pd
from tqdm.autonotebook import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import gc
import logging
import gensim
import jieba
tqdm.pandas()
os.environ['PYTHONHASHSEED'] = '0'
# 显卡使用（如没显卡需要注释掉）
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
np.random.seed(1024)
rn.seed(1024)
tf.random.set_seed(1024)

In [29]:
train_data['text_1'] = train_data['text_1'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
train_data['text_2'] = train_data['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
# train_data['text_2'] = train_data['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
train_data.head(5)

100%|██████████| 27154/27154 [06:03<00:00, 74.65it/s]


,text_1,text_2,label_1,label_2,label_3,label_4
0,中银 保本 理财 - 人民币 按期 开放 理财产品 _ 中国银行 股份 有限公司 广州 东山...,引言 [ ' 证券 代码 ： 600728 证券 简称 ： 佳 都 科技 公告 编号...,9,1,0,0
1,中银 保本 理财 - 人民币 按期 开放 理财产品 _ 中国银行 股份 有限公司 广州 东山...,引言 [ ' 证券 代码 ： 600728 证券 简称 ： 佳 都 科技 公告 编号...,9,1,0,0
2,与 利率 挂钩 的 结构性 产品 _ 中国民生银行 股份 有限公司 _ 西藏 诺迪康 药业 ...,引言 [ ' 证券 代码 ： 600211 证券 简称 ： 西藏药业 公告 编号 ：...,9,0,0,0
3,广发 银行 “ 薪 加薪 ” 16 号 XJXCKJ2578 _ 广发 银行 股份 有限公司...,引言 [ ' 证券 代码 ： 002171 证券 简称 ： 楚江 新材 公告 编号 ...,9,1,2,0
4,兴业银行 “ 金 雪球 - 优悦 ” 保本 开放式 人民币 理财产品 ( 2M ) _ 兴业...,引言 [ ' 证券 代码 ： 002171 证券 简称 ： 楚江 新材 公告 编号 ...,9,1,0,0


In [30]:
### Tokenizer 序列化文本
def set_tokenizer(docs, split_char=' ', max_len=100):
    '''
    输入
    docs:文本列表
    split_char:按什么字符切割
    max_len:截取的最大长度
    
    输出
    X:序列化后的数据
    word_index:文本和数字对应的索引
    '''
    tokenizer = Tokenizer(lower=False, char_level=False, split=split_char)
    tokenizer.fit_on_texts(docs)
    X = tokenizer.texts_to_sequences(docs)
    maxlen = max_len
    X = pad_sequences(X, maxlen=maxlen, value=0)
    word_index=tokenizer.word_index
    return X, word_index, tokenizer

### 做embedding 这里采用word2vec 可以换成其他例如（glove词向量）
def trian_save_word2vec(docs, embed_size=300, save_name='w2v.txt', split_char=' '):
    '''
    输入
    docs:输入的文本列表
    embed_size:embed长度
    save_name:保存的word2vec位置
    
    输出
    w2v:返回的模型
    '''
    input_docs = []
    for i in docs:
        input_docs.append(i.split(split_char))
    logging.basicConfig(
    format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(input_docs, size=embed_size, sg=1, window=8, seed=1017, workers=24, min_count=1, iter=10)
    w2v.save(save_name)
    print("w2v model done")
    return w2v

# 得到embedding矩阵
def get_embedding_matrix(word_index, embed_size=300, Emed_path="w2v_300.txt"):
    embeddings_index = Word2Vec.load(Emed_path)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector  
    print("null cnt",count)
    return embedding_matrix

# 得到fasttext矩阵
def load_fasttext(word_index, path):  
    count=0
    null_list=[]
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(path, encoding='utf-8') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words =  len(word_index)+1
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            null_list.append(word)
            count+=1
    print("null cnt:",count)
    return embedding_matrix

def get_embedding_matrix_txt(word_index,embed_size=200,Emed_path="w2v_300.txt"):
    embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(
        Emed_path, binary=False)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print("null cnt",count)
    return embedding_matrix

### 2.训练得到word2vec

In [31]:
text_1_list = np.unique(train_data['text_1'])
# text_2_list = np.unique(train_data['text_3'])
text_3_list = np.unique(train_data['text_2'])

print('开始序列化')
x1, index_1, token_1 = set_tokenizer(train_data['text_1'], split_char=' ', max_len=60)
x3, index_3, token_3 = set_tokenizer(train_data['text_2'], split_char=' ', max_len=600)

print('序列化完成')
gc.collect()

trian_save_word2vec(text_1_list, save_name='models/w2v_300_1.txt', split_char=' ')
gc.collect()
trian_save_word2vec(text_3_list, save_name='models/w2v_300_3.txt', split_char=' ')
gc.collect()

# 得到emb矩阵
emb1 = get_embedding_matrix(index_1, Emed_path='models/w2v_300_1.txt')
emb3 = get_embedding_matrix(index_3, Emed_path='models/w2v_300_3.txt')
gc.collect()

开始序列化
序列化完成


0

8 more threads
2020-09-12 12:47:43,883:INFO:worker thread finished; awaiting finish of 17 more threads
2020-09-12 12:47:43,886:INFO:worker thread finished; awaiting finish of 16 more threads
2020-09-12 12:47:43,899:INFO:worker thread finished; awaiting finish of 15 more threads
2020-09-12 12:47:43,907:INFO:worker thread finished; awaiting finish of 14 more threads
2020-09-12 12:47:43,915:INFO:worker thread finished; awaiting finish of 13 more threads
2020-09-12 12:47:43,920:INFO:worker thread finished; awaiting finish of 12 more threads
2020-09-12 12:47:43,922:INFO:worker thread finished; awaiting finish of 11 more threads
2020-09-12 12:47:43,928:INFO:worker thread finished; awaiting finish of 10 more threads
2020-09-12 12:47:43,934:INFO:worker thread finished; awaiting finish of 9 more threads
2020-09-12 12:47:43,937:INFO:worker thread finished; awaiting finish of 8 more threads
2020-09-12 12:47:43,949:INFO:worker thread finished; awaiting finish of 7 more threads
2020-09-12 12:47:43,

0

s/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:22,044:INFO:EPOCH 6 - PROGRESS: at 9.56% examples, 213800 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:23,085:INFO:EPOCH 6 - PROGRESS: at 16.96% examples, 247457 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:24,161:INFO:EPOCH 6 - PROGRESS: at 23.23% examples, 252949 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:25,168:INFO:EPOCH 6 - PROGRESS: at 31.38% examples, 266538 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:26,170:INFO:EPOCH 6 - PROGRESS: at 36.05% examples, 263499 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:27,184:INFO:EPOCH 6 - PROGRESS: at 40.33% examples, 265560 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:28,187:INFO:EPOCH 6 - PROGRESS: at 45.27% examples, 272918 words/s, in_qsize 46, out_qsize 1
2020-09-12 12:49:29,221:INFO:EPOCH 6 - PROGRESS: at 52.83% examples, 273797 words/s, in_qsize 47, out_qsize 0
2020-09-12 12:49:30,287:INFO:EPOCH 6 - PROGRESS: at 60.49% examples, 274831 words/s, in_qsi

0

2020-09-12 12:50:45,285:INFO:loading Word2Vec object from models/w2v_300_1.txt
2020-09-12 12:50:45,556:INFO:loading wv recursively from models/w2v_300_1.txt.wv.* with mmap=None
2020-09-12 12:50:45,558:INFO:setting ignored attribute vectors_norm to None
2020-09-12 12:50:45,559:INFO:loading vocabulary recursively from models/w2v_300_1.txt.vocabulary.* with mmap=None
2020-09-12 12:50:45,561:INFO:loading trainables recursively from models/w2v_300_1.txt.trainables.* with mmap=None
2020-09-12 12:50:45,562:INFO:setting ignored attribute cum_table to None
2020-09-12 12:50:45,564:INFO:loaded models/w2v_300_1.txt
100%|██████████| 8360/8360 [00:00<00:00, 44751.46it/s]
2020-09-12 12:50:45,805:INFO:loading Word2Vec object from models/w2v_300_3.txt
null cnt 5
2020-09-12 12:50:46,383:INFO:loading wv recursively from models/w2v_300_3.txt.wv.* with mmap=None
2020-09-12 12:50:46,384:INFO:setting ignored attribute vectors_norm to None
2020-09-12 12:50:46,385:INFO:loading vocabulary recursively from model

9

# 构建抽取模型

#### 1.网络结构

In [32]:
from keras.initializers import *

def model_conv(emb1, emb3):
    '''
    注意这个inputs
    seq1、seq2分别是两个输入
    是否做emb可选可不选，
    这个就是我们之前训练已经得到的用于embedding的（embedding_matrix1， embedding_matrix2）
    '''
    K.clear_session()

    emb_layer_1 = Embedding(
        input_dim=emb1.shape[0],
        output_dim=emb1.shape[1],
        weights=[emb1],
        input_length=30,
        trainable=False
    )
    
    emb_layer_3 = Embedding(
        input_dim=emb3.shape[0],
        output_dim=emb3.shape[1],
        weights=[emb3],
        input_length=600,
        trainable=False
    )
    
    
    seq1 = Input(shape=(30,))
    seq3 = Input(shape=(600,))    
    
    x1 = emb_layer_1(seq1)
    x3 = emb_layer_3(seq3)
    
    sdrop=SpatialDropout1D(rate=0.2)

    x1 = sdrop(x1)
    x3 = sdrop(x3)
     
    x = Dropout(0.2)(Bidirectional(GRU(128, return_sequences=True))(x1))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    
    x = Dropout(0.2)(Bidirectional(GRU(128, return_sequences=True))(x3))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_3 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    
    
    x = Multiply()([merged_1, merged_3])
    
    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(1000)(x))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))
    pred_1 = Dense(10, activation='softmax')(x)
    pred_2 = Dense(3, activation='softmax')(x)
    pred_3 = Dense(3, activation='softmax')(x)
    pred_4 = Dense(2, activation='softmax')(x)
    model = Model(inputs=[seq1, seq3], outputs=[pred_1, pred_2, pred_3, pred_4])
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001),metrics=["accuracy"])
    return model
gc.collect()

0

In [33]:
model = model_conv(emb1, emb3)
model.summary()
l1 = to_categorical(train_data['label_1'], 10)
l2 = to_categorical(train_data['label_2'], 3)
l3 = to_categorical(train_data['label_3'], 3)
l4 = to_categorical(train_data['label_4'], 2)
model.fit([x1, x3],[l1, l2, l3, l4], batch_size=256, epochs=8, verbose=1, shuffle=True)
# model.load_weights('models\lstm_model.h5')

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 600)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      2508300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 600, 300)     6548100     input_2[0][0]                    
_______________________________________________________________________________________

In [35]:
#保存权重
model.save_weights('models/lstm_valid_text_model.h5')

#### 2.保存结果

In [36]:
# 预测验证集
val_df["sample_id"]=val_df["sample_id"].astype(str)
val_result_for_pred = pd.merge(val_result, val_df, on='sample_id', how='left')
val_result_for_pred['text_1'] = val_result_for_pred['理财产品名称'].astype(str) + '_' + val_result_for_pred['产品发行方名称'].astype(str)
val_result_for_pred['text_2'] = val_result_for_pred['text'].astype(str)

val_result_for_pred['text_1'] = val_result_for_pred['text_1'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
val_result_for_pred['text_2'] = val_result_for_pred['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))

x1 = token_1.texts_to_sequences(val_result_for_pred['text_1'])
x1 = pad_sequences(x1, maxlen=30, value=0)
x3 = token_3.texts_to_sequences(val_result_for_pred['text_2'])
x3 = pad_sequences(x3, maxlen=600, value=0)
pred_result = model.predict([x1, x3], batch_size=1024, verbose=1)
pred_1 = label_1.inverse_transform(np.argmax(pred_result[0], axis=1))
pred_2 = label_2.inverse_transform(np.argmax(pred_result[1], axis=1))
pred_3 = label_3.inverse_transform(np.argmax(pred_result[2], axis=1))
pred_4 = label_4.inverse_transform(np.argmax(pred_result[3], axis=1))


NameError: name 'val_result' is not defined

In [37]:
val_result['理财类型'] = pred_1
val_result['资金来源'] = pred_2
val_result['实际购买公司和上市公司关系'] = pred_3
val_result['买卖方是否有关联关系'] = pred_4

NameError: name 'pred_1' is not defined

#### 3.离线验证评估

In [38]:
# 计算线下f1
# R（召回率）=抽取正确的记录数量/（抽取正确的目标数量+漏抽取的记录数量）
# P（准确率）=抽取正确的记录数量/（抽取错误的记录数量+抽取正确的记录数量）
def get_F1(val_pred, val_true):
    val_pred = list(val_pred)
    val_true = list(val_true)
    curr = list(set(val_pred).intersection(set(val_true)))
    R = len(curr)/len(val_true)
    P = len(curr)/len(val_pred)
    return 2*P*R/(P+R)

train_outputs["sample_id"]=train_outputs["sample_id"].astype(str)
r = pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left')
val_true = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['理财类型'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str) + r['资金来源'].astype(str) + r['实际购买公司名称'].astype(str) + r['实际购买公司和上市公司关系'].astype(str) + r['买卖方是否有关联关系'].astype(str) + r['公告日期'].astype(str)

r = val_result
val_pred = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['理财类型'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str) + r['资金来源'].astype(str) + r['实际购买公司名称'].astype(str) + r['实际购买公司和上市公司关系'].astype(str) + r['买卖方是否有关联关系'].astype(str) + r['公告日期'].astype(str)

score = get_F1(val_pred, val_true)
score


NameError: name 'val_result' is not defined

In [39]:
val_true_file=pl.Path("results/val_true.csv")
val_pred_file=pl.Path("results/val_pred.csv")

tr_true_file=pl.Path("results/tr_true.csv")
val_result.copy().sort_values(by="sample_id").sort_index(axis=1).to_csv(val_pred_file,index=None)
pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left').sort_index(axis=1).sort_values(by="sample_id").to_csv(val_true_file,index=None)
pd.merge(train_df[['sample_id']], train_outputs, on='sample_id', how='left').sort_index(axis=1).sort_values(by="sample_id").to_csv(tr_true_file,index=None)

NameError: name 'val_result' is not defined

#### 4.最终输出结果

In [53]:
test_result_data=pl.Path("text_test_2210_result.xlsx")
test_result=pd.read_excel(test_result_data)
test_result["sample_id"]=test_result["sample_id"].astype(str)

In [54]:
# 预测测试集
test_df["sample_id"]=test_df["sample_id"].astype(str)
test_result_for_pred = pd.merge(test_result, test_df, on='sample_id', how='left')
test_result_for_pred['text_1'] = test_result_for_pred['理财产品名称'].astype(str) + '_' + test_result_for_pred['产品发行方名称'].astype(str)+"_"+train_lstm_input['实际购买公司名称'].astype(str)
test_result_for_pred['text_2'] = test_result_for_pred['text'].astype(str)

test_result_for_pred['text_1'] = test_result_for_pred['text_1'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
test_result_for_pred['text_2'] = test_result_for_pred['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))

x1 = token_1.texts_to_sequences(test_result_for_pred['text_1'])
x1 = pad_sequences(x1, maxlen=30, value=0)
x3 = token_3.texts_to_sequences(test_result_for_pred['text_2'])
x3 = pad_sequences(x3, maxlen=600, value=0)
pred_result = model.predict([x1, x3], batch_size=1024, verbose=1)
pred_1 = label_1.inverse_transform(np.argmax(pred_result[0], axis=1))
pred_2 = label_2.inverse_transform(np.argmax(pred_result[1], axis=1))
pred_3 = label_3.inverse_transform(np.argmax(pred_result[2], axis=1))
pred_4 = label_4.inverse_transform(np.argmax(pred_result[3], axis=1))

test_result['理财类型'] = pred_1
test_result['资金来源'] = pred_2
test_result['实际购买公司和上市公司关系'] = pred_3
test_result['买卖方是否有关联关系'] = pred_4

6/6 [==============================] - 43s 7s/step


In [55]:
# test_result.to_csv('results/re_lstm_base.csv',encoding="utf8", index=False)
test_result.to_excel('results/re_lstm_base2218.xlsx',encoding="gb18030", index=False)

files_num=int(test_df.shape[0]/4)
test_time["sample_id"]=test_time["sample_id"].astype(str)

# test_time.shape
# test_time.dropna(subset=["公告日期"]).shape
import random
for i in range(1,20):
    random_seed=random.randint(0,10000)
    random_test_df = test_df.sample(frac=1, random_state=random_seed)
    change_df=random_test_df[:files_num]
    stable_df=random_test_df[files_num:]
    random_test_time=pd.merge(change_df,test_time,on=["sample_id"])
    random_test_time.shape
    random_test_time=random_test_time.dropna(subset=["公告日期"])
    random_test_time.shape
    random_test_time["公告日期"]=random_test_time["公告日期"].map(lambda x:datetime.datetime.strftime(datetime.datetime.strptime(str(x.replace("\r","")), '%Y-%m-%d'),'%Y-%m-%d'))


In [42]:
sample_id=4663
# sample_id=7123
tabel=val_df[val_df["sample_id"]==sample_id]["tabel"].iloc[0]
table_result,start_rows_list,first_line_list,product_df_list=row_combine(sample_id,tabel)
index=0
table_result[index]
product_df_list_ele=product_df_list[index].reset_index(drop=True)
product_df_list_ele
# product_df_list_ele
each_sum_rows=get_each_product_row(table_result[index],product_df_list_ele.reset_index(drop=True))

each_sum_rows2=result_matrix[result_matrix["sample_id"]==sample_id]["product_df"].iloc[0]
# # each_sum_rows
each_sum_rows

IndexError: single positional indexer is out-of-bounds